In [ ]:
import pandas as pd
#import numpy as np 과제에서 numpy를 사용하지 않기에 불러오지않음.

crime_excel= '관서별 5대범죄 발생 및 검거.xlsx'

# 엑셀파일을 가져와서 데이터프레임으로 만들기
df = pd.read_excel(crime_excel)
# 데이터프레임 출력
df

In [ ]:
# 각 경찰서(`관서명`)를 해당 구 이름으로 매핑하여 '구별'이라는 새로운 column을 생성하고, DataFrame을 출력해주세요.
# 매칭되지 않는 경찰서명에 대해서는 기본값 `'구 없음'`을 할당합니다.

# 관서명과 구 이름 간의 매핑 딕셔너리
mapping_dict = {
    '서대문서': '서대문구', '수서서': '강남구', '강서서': '강서구', '서초서': '서초구',
    '서부서': '은평구', '중부서': '중구', '종로서': '종로구', '남대문서': '중구',
    '혜화서': '종로구', '용산서': '용산구', '성북서': '성북구', '동대문서': '동대문구',
    '마포서': '마포구', '영등포서': '영등포구', '성동서': '성동구', '동작서': '동작구',
    '광진서': '광진구', '강북서': '강북구', '금천서': '금천구', '중랑서': '중랑구',
    '강남서': '강남구', '관악서': '관악구', '강동서': '강동구', '종암서': '성북구', 
    '구로서': '구로구', '양천서': '양천구', '송파서': '송파구', '노원서': '노원구', 
    '방배서': '서초구', '은평서': '은평구', '도봉서': '도봉구'
}

# '관서명' 칼럼을 '구별' 칼럼으로 매핑
df['구별'] = df['관서명'].map(mapping_dict).fillna('구 없음')

df

In [ ]:
# pivot_table 을 사용하여 관서별 데이터를 구별 데이터로 변경하고, 같은 구의 경우에는 sum을 적용하여 더해주세요. (index : 관서명 -> 구별)
# pivot_table을 사용하여 구별로 데이터 집

# 구별과 관서명을 제외한 모든 컬럼 사용
df_by_district = pd.pivot_table(df,
                               index='구별',
                               values=df.columns.drop(['구별', '관서명']),
                               aggfunc='sum')

df_by_district

In [ ]:
# '구 없음' 행을 삭제
df_by_district = df_by_district.drop('구 없음')

df_by_district

In [ ]:
# 각 범죄 유형별 검거율 계산 및 새로운 컬럼 추가
df_by_district['강간검거율'] = (df_by_district['강간(검거)'] / df_by_district['강간(발생)']) * 100
df_by_district['강도검거율'] = (df_by_district['강도(검거)'] / df_by_district['강도(발생)']) * 100
df_by_district['살인검거율'] = (df_by_district['살인(검거)'] / df_by_district['살인(발생)']) * 100
df_by_district['절도검거율'] = (df_by_district['절도(검거)'] / df_by_district['절도(발생)']) * 100
df_by_district['폭력검거율'] = (df_by_district['폭력(검거)'] / df_by_district['폭력(발생)']) * 100
df_by_district['검거율'] = (df_by_district['소계(검거)'] / df_by_district['소계(발생)']) * 100

df_by_district

In [ ]:
# 여러 컬럼을 del로 삭제
del df_by_district['강간(검거)']
del df_by_district['강도(검거)']
del df_by_district['살인(검거)']
del df_by_district['절도(검거)']
del df_by_district['폭력(검거)']
del df_by_district['소계(발생)']
del df_by_district['소계(검거)']

df_by_district

In [ ]:
# 컬럼명 변경
df_by_district = df_by_district.rename(columns={
    '강간(발생)': '강간',
    '강도(발생)': '강도',
    '살인(발생)': '살인',
    '절도(발생)': '절도',
    '폭력(발생)': '폭력'
})

df_by_district

In [ ]:
pop_url = 'https://drive.google.com/uc?id=1fsyI1vM8wNtu-UZhlQ2XvLSRl3tpfYYJ' 

# 인구 데이터 파일 불러오기
pop_df = pd.read_csv(pop_url, index_col='구별')

pop_df


In [ ]:
# df_by_district와 pop_df를 join으로 병합
df_merged = df_by_district.join(pop_df)

df_merged


In [ ]:
# 검거율 기준으로 오름차순 정렬
df_merged_sorted = df_merged.sort_values(by='검거율', ascending=True)

df_merged_sorted